In [1]:
# Colabis kasutamiseks
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/thesis/')

!pip install estnltk==1.7.4
!pip install evaluate
!pip install seqeval
!pip install nervaluate

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling

In [2]:
from data_processing import DatasetProcessor
from bert_data_processing import BERTDataProcessor
from bert_evaluator import BERTEvaluator
from bert_trainer import BERTTrainer
ALL_TAGS = DatasetProcessor.ALL_TAGS
TAG2IDX = DatasetProcessor.TAG2IDX
IDX2TAG = DatasetProcessor.IDX2TAG

INFO:datasets:PyTorch version 2.5.1+cu124 available.
INFO:datasets:Polars version 1.9.0 available.
INFO:datasets:Duckdb version 1.1.3 available.
INFO:datasets:TensorFlow version 2.18.0 available.
INFO:datasets:JAX version 0.4.33 available.


INFO:config.py:54: PyTorch version 2.5.1+cu124 available.
INFO:config.py:66: Polars version 1.9.0 available.
INFO:config.py:77: Duckdb version 1.1.3 available.
INFO:config.py:112: TensorFlow version 2.18.0 available.
INFO:config.py:125: JAX version 0.4.33 available.


In [8]:
def train_model(model_name, dataset_name, epochs=3, batch_size=16):
    if dataset_name.lower() == 'combined':
      ewt_processor = DatasetProcessor('ewt', from_json=True)
      edt_processor = DatasetProcessor('edt', from_json=True)
      ewt_dataset = ewt_processor.load_or_process()
      edt_dataset = edt_processor.load_or_process()
      dataset = DatasetProcessor.combine_datasetdicts(ewt_dataset, edt_dataset)
    elif dataset_name.lower() in ['ewt', 'edt']:
      processor = DatasetProcessor(dataset_name, from_json=True)
      dataset = processor.load_or_process()

    print(f'{dataset_name.upper()} andmestik laetud')
    bert_processor = BERTDataProcessor(model_name)
    evaluator = BERTEvaluator(all_tags=ALL_TAGS)

    tokenized_dataset = bert_processor.tokenize_dataset(dataset)

    trainer = BERTTrainer(model_name=model_name, idx2tag=IDX2TAG, tag2idx=TAG2IDX, evaluator=evaluator)

    model, model_trainer = trainer.finetune_model(processor=bert_processor, tokenized_dataset=tokenized_dataset, epochs=epochs, batch_size=batch_size, early_stop_patience=3, output_dir=f'./results/{model_name.split("/")[1]}/{dataset_name}')

    results = evaluator.evaluate_and_print(tokenized_dataset['test'], model_trainer)

    return model, model_trainer, results

def train_all():
  dataset_configs = {
        'ewt': {'epochs': 10},
        'edt': {'epochs': 5},
        'combined': {'epochs': 3}
    }

  model_names = ["tartuNLP/EstRoBERTa", "tartuNLP/EstBERT"]
  results_dict = {}

  for model_name in model_names:
    model_results = {}
    for dataset_name, config in dataset_configs.items():
      model, model_trainer, results = train_model(model_name, dataset_name, epochs=config['epochs'])
      model_results[dataset_name] = results
    results_dict[model_name] = model_results

  return results_dict

In [9]:
def main():
    results = train_all()
    try:
      for model_name, model_results in results.items():
        for dataset_name, dataset_results in model_results.items():
          print(f"{model_name}, {dataset_name}")
          print(dataset_results)
          print()
    except:
      pass

In [10]:
main()

EWT dataset loaded


tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Eve precision,Eve recall,Eve f1,Eve number,Gep precision,Gep recall,Gep f1,Gep number,Loc precision,Loc recall,Loc f1,Loc number,Muu precision,Muu recall,Muu f1,Muu number,Org precision,Org recall,Org f1,Org number,Per precision,Per recall,Per f1,Per number,Prod precision,Prod recall,Prod f1,Prod number,Unk precision,Unk recall,Unk f1,Unk number
1,0.092700,0.133657,0.729124,0.709000,0.718921,0.966568,0.500000,0.397059,0.442623,68,0.589916,0.780000,0.671770,450,0.795455,0.542636,0.645161,645,0.000000,0.000000,0.000000,12,0.478941,0.768340,0.590067,518,0.937037,0.871662,0.903168,2322,0.505115,0.432640,0.466077,913,0.000000,0.000000,0.000000,72
2,0.039900,0.147840,0.731600,0.731600,0.731600,0.968088,0.382979,0.529412,0.444444,68,0.601562,0.684444,0.640333,450,0.705785,0.662016,0.683200,645,0.000000,0.000000,0.000000,12,0.542113,0.683398,0.604611,518,0.935181,0.900947,0.917745,2322,0.490545,0.483023,0.486755,913,0.000000,0.000000,0.000000,72
3,0.020000,0.171511,0.740862,0.725600,0.733151,0.968768,0.333333,0.367647,0.349650,68,0.580000,0.773333,0.662857,450,0.745597,0.590698,0.659170,645,0.000000,0.000000,0.000000,12,0.542222,0.706564,0.613579,518,0.936572,0.896641,0.916172,2322,0.524631,0.466594,0.493913,913,0.000000,0.000000,0.000000,72


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[2025-02-12 13:58:10] tartuNLP/EstBERT treenimine lõpetatud
Kokku kulus: 888.35 sekundit (0.25 tundi)
Hindan testandmestikul..


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seqeval tulemused
EVE {'precision': 0.48091603053435117, 'recall': 0.44680851063829785, 'f1': 0.4632352941176471, 'number': 141}
GEP {'precision': 0.6, 'recall': 0.765375854214123, 'f1': 0.6726726726726726, 'number': 439}
LOC {'precision': 0.7223340040241448, 'recall': 0.7237903225806451, 'f1': 0.7230614300100703, 'number': 496}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}
ORG {'precision': 0.7300546448087432, 'recall': 0.6958333333333333, 'f1': 0.7125333333333334, 'number': 960}
PER {'precision': 0.9399938137952366, 'recall': 0.8880771478667446, 'f1': 0.9132982719759579, 'number': 3422}
PROD {'precision': 0.4866023579849946, 'recall': 0.47689075630252103, 'f1': 0.4816976127320955, 'number': 952}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.7846546498644121
overall_recall 0.7664381427235899
overall_f1 0.7754394261842831
overall_accuracy 0.9703505703234707

Nervaluate tulemused
Strict {'correct': 4919, 'incorrect': 967, 'partial': 0,